In [2]:
import torch 
import torch.nn as nn
import nltk
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from transformers import BertTokenizer, BertModel
from tqdm import tqdm

In [3]:
import pandas as pd

data = pd.read_table('fra.txt', header= None)

In [4]:
data.rename(columns= {0: 'English', 1: 'French', 2: 'Citation'}, inplace= True)

In [5]:
pattern = r"[!'#$%&()*+,-./:;<=>?@[\]^`{|}~“”‘’«»‹›„‚–—…·•¡¿’\"\']"

eng_sent, french_sent = [], []

for e in range(len(data['English'])):
    eng_sent.append(re.sub(pattern, "", data['English'][e]))
    french_sent.append(re.sub(pattern, "", data['French'][e]))

In [6]:
print(len(eng_sent))
print(len(french_sent))

229803
229803


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
eng_token = [tokenizer.encode(text, add_special_tokens= True) for text in eng_sent]
fren_token = [tokenizer.encode(text, add_special_tokens= True) for text in french_sent]

In [9]:
model = BertModel.from_pretrained('bert-base-uncased')


In [10]:
def text_embedding(text_token):
    tokens_tensor = torch.tensor([text_token])

    with torch.no_grad():
        output = model(tokens_tensor)
        embeddings = output.last_hidden_state
    return embeddings

In [11]:
eng_embedding = []
for token in tqdm(eng_token, "Embedding", colour= "green"):
    eng_embedding.append(text_embedding(token))

len(eng_embedding)

Embedding:  71%|███████   | 162079/229803 [1:59:42<50:01, 22.57it/s]  


KeyboardInterrupt: 

In [ ]:
fren_embedding = []
for token in tqdm(fren_token, "Embedding", colour= "green"):
    fren_embedding.append(text_embedding(token))

len(fren_embedding)

Embedding: 100%|██████████| 229803/229803 [1:55:54<00:00, 33.04it/s]  


229803

In [ ]:
for i in tqdm(range(len(fren_embedding)), "Saving"):
    file_name = "French_embedding_" + str(i) + ".pt"
    torch.save(fren_embedding[i], file_name)

NameError: name 'tqdm' is not defined

In [ ]:
for i in tqdm(range(len(eng_embedding)), "Saving"):
    file_name = "English_embedding_" + str(i) + ".pt"
    torch.save(eng_embedding[i], file_name)